In [0]:
-- ============================================================================
-- ETL Mapping: ADS_RDS_EVENT_TYPES_MEP (Pure SQL Version)
-- Converted from Databricks Python Notebook to SQL Warehouse compatible
-- Generated date: 2025-09-22
-- ============================================================================

-- Conversion notes:
-- - Removed Python dependencies (dbutils, spark.sql, etc.)
-- - Converted to pure SQL with parameters
-- - Compatible with Databricks SQL Warehouse
-- - Uses SQL variables for dynamic values

-- ============================================================================
-- STEP 1: Set Parameters and Variables
-- ============================================================================

-- Define parameters (these can be passed from job parameters or set here)
DECLARE OR REPLACE VARIABLE p_load_date STRING DEFAULT '2025-08-14';
DECLARE OR REPLACE VARIABLE p_process_key STRING DEFAULT '13173833';
DECLARE OR REPLACE VARIABLE map_id STRING DEFAULT 'ADS_RDS_EVENT_TYPES_MEP';
DECLARE OR REPLACE VARIABLE dif_table_name STRING DEFAULT 'gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MEP_ADS_MAP_SCD_DIFF';

-- Get the maximum EVETP_KEY from Target table
DECLARE OR REPLACE VARIABLE max_key BIGINT DEFAULT (
    SELECT COALESCE(MAX(evetp_key), 0) 
    FROM gap_catalog.ads_owner.event_types
);

-- Display current maximum key for logging
SELECT 'Current maximum EVETP_KEY: ' || max_key as info_message;

-- ============================================================================
-- STEP 2: Truncate XC Table
-- ============================================================================

TRUNCATE TABLE gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MEP;

-- ============================================================================
-- STEP 3: Fill XC Table
-- ============================================================================

INSERT INTO gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MEP 
WITH C_0FILTERALQDR4RF0LLIP1736450PD5NP4 AS (
    SELECT
        RDS_MEPOPERATIONTYPE.id AS ID,
        RDS_MEPOPERATIONTYPE.value AS VALUE
    FROM gap_catalog.ads_etl_owner.DLK_ADS_LOV_RDS_MEPOPERATIONTYPE RDS_MEPOPERATIONTYPE
    WHERE RDS_MEPOPERATIONTYPE.sys = 'Brasil'
        AND RDS_MEPOPERATIONTYPE.lang = 'CZ'
        AND CAST(from_utc_timestamp(SYS_EFFECTIVE_DATE, 'Europe/Prague') AS DATE) = p_load_date
)
SELECT /*+no hint*/
    /*seznam vkladanych nebo updatovanych sloupcu bez SK, auditnich atributu a deleted flagu*/
    FILTER_A.ID AS EVETP_SOURCE_ID,
    'RDS' AS EVETP_SOURCE_SYSTEM_ID,
    'RDS_MEPOPERATIONTYPE' AS EVETP_SOURCE_SYS_ORIGIN,
    FILTER_A.VALUE AS EVETP_DESC,
    FILTER_A.ID AS EVETP_EVENT_TABLE_NAME,
    'XNA' AS EVETP_TARGET,
    'XNA' AS EVETP_DATA_PATH,
    'N' AS EVETP_DELETED_FLAG
FROM C_0FILTERALQDR4RF0LLIP1736450PD5NP4 FILTER_A
WHERE 1=1;

-- ============================================================================
-- STEP 4: Cleanup and Create DIFF Table
-- ============================================================================

-- Use IDENTIFIER to handle dynamic table name
DROP TABLE IF EXISTS IDENTIFIER(dif_table_name);

CREATE TABLE IDENTIFIER(dif_table_name) (
    tech_del_flg CHAR(1),
    tech_new_rec CHAR(1),
    tech_rid VARCHAR(255),
    EVETP_KEY INTEGER,
    EVETP_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
    EVETP_SOURCE_ID VARCHAR(400),
    EVETP_SOURCE_SYSTEM_ID VARCHAR(120),
    EVETP_SOURCE_SYS_ORIGIN VARCHAR(120),
    EVETP_DESC VARCHAR(4000),
    EVETP_EVENT_TABLE_NAME VARCHAR(120),
    EVETP_TARGET VARCHAR(120),
    EVETP_DATA_PATH VARCHAR(120)
);

-- Note: The identity column will start from 1, we'll handle the max_key offset in the final insert

-- ============================================================================
-- STEP 5: Populate DIFF Table - New/Updated Records
-- ============================================================================

INSERT INTO IDENTIFIER(dif_table_name) (
    tech_del_flg,
    tech_new_rec,
    tech_rid,
    EVETP_KEY,
    EVETP_SOURCE_ID,
    EVETP_SOURCE_SYSTEM_ID,
    EVETP_SOURCE_SYS_ORIGIN,
    EVETP_DESC,
    EVETP_EVENT_TABLE_NAME,
    EVETP_TARGET,
    EVETP_DATA_PATH
)
SELECT 
    'N' AS tech_del_flg, 
    CASE WHEN trg.EVETP_SOURCE_ID IS NULL THEN 'Y' ELSE 'N' END AS tech_new_rec,
    trg.rid AS tech_rid,
    trg.EVETP_KEY, 
    src.EVETP_SOURCE_ID, 
    src.EVETP_SOURCE_SYSTEM_ID, 
    src.EVETP_SOURCE_SYS_ORIGIN, 
    src.EVETP_DESC, 
    src.EVETP_EVENT_TABLE_NAME, 
    src.EVETP_TARGET, 
    src.EVETP_DATA_PATH
FROM (
    SELECT /*+ full(xc) */
        EVETP_SOURCE_ID, 
        EVETP_SOURCE_SYSTEM_ID, 
        EVETP_SOURCE_SYS_ORIGIN, 
        EVETP_DESC, 
        EVETP_EVENT_TABLE_NAME, 
        EVETP_DELETED_FLAG, 
        EVETP_TARGET, 
        EVETP_DATA_PATH
    FROM gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MEP xc
    WHERE EVETP_SOURCE_SYS_ORIGIN = 'RDS_MEPOPERATIONTYPE'
) src 
LEFT JOIN (
    SELECT 
        evetp_key||'.'||evetp_valid_from AS rid, 
        t.* 
    FROM gap_catalog.ads_owner.EVENT_TYPES t
    WHERE EVETP_CURRENT_FLAG = 'Y'
        AND EVETP_VALID_TO = to_date('01013000','ddMMyyyy')
        AND EVETP_SOURCE_SYS_ORIGIN = 'RDS_MEPOPERATIONTYPE'
) trg
ON trg.EVETP_SOURCE_ID = src.EVETP_SOURCE_ID
    AND trg.EVETP_SOURCE_SYSTEM_ID = src.EVETP_SOURCE_SYSTEM_ID
    AND trg.EVETP_SOURCE_SYS_ORIGIN = src.EVETP_SOURCE_SYS_ORIGIN
    AND trg.EVETP_VALID_TO = to_date('30000101','yyyyMMdd') 
WHERE (
    decode(src.EVETP_DESC, trg.EVETP_DESC, 1, 0) = 0 OR
    decode(src.EVETP_EVENT_TABLE_NAME, trg.EVETP_EVENT_TABLE_NAME, 1, 0) = 0 OR
    decode(src.EVETP_DELETED_FLAG, trg.EVETP_DELETED_FLAG, 1, 0) = 0 OR
    decode(src.EVETP_TARGET, trg.EVETP_TARGET, 1, 0) = 0 OR
    decode(src.EVETP_DATA_PATH, trg.EVETP_DATA_PATH, 1, 0) = 0 OR 
    trg.EVETP_SOURCE_ID IS NULL
);

-- ============================================================================
-- STEP 6: Populate DIFF Table - Deleted Records
-- ============================================================================

INSERT INTO IDENTIFIER(dif_table_name) (
    tech_del_flg,
    tech_new_rec,
    tech_rid,
    EVETP_KEY,
    EVETP_SOURCE_ID,
    EVETP_SOURCE_SYSTEM_ID,
    EVETP_SOURCE_SYS_ORIGIN,
    EVETP_DESC,
    EVETP_EVENT_TABLE_NAME,
    EVETP_TARGET,
    EVETP_DATA_PATH
)
SELECT 
    'Y' AS tech_del_flg, 
    'N' AS tech_new_rec, 
    trg.rid AS tech_rid, 
    trg.EVETP_KEY, 
    trg.EVETP_SOURCE_ID, 
    trg.EVETP_SOURCE_SYSTEM_ID, 
    trg.EVETP_SOURCE_SYS_ORIGIN, 
    trg.EVETP_DESC, 
    trg.EVETP_EVENT_TABLE_NAME, 
    trg.EVETP_TARGET, 
    trg.EVETP_DATA_PATH
FROM (
    SELECT 
        evetp_key||'.'||evetp_valid_from AS rid, 
        t.* 
    FROM gap_catalog.ads_owner.EVENT_TYPES t
    WHERE EVETP_CURRENT_FLAG = 'Y' 
        AND EVETP_DELETED_FLAG = 'N'
        AND EVETP_VALID_TO = to_date('01013000','ddMMyyyy')
        AND EVETP_SOURCE_SYS_ORIGIN = 'RDS_MEPOPERATIONTYPE'
) trg 
LEFT JOIN (
    SELECT /*+ full(xc) */
        EVETP_SOURCE_ID, 
        EVETP_SOURCE_SYSTEM_ID, 
        EVETP_SOURCE_SYS_ORIGIN, 
        to_date('30000101','yyyyMMdd') AS EVETP_VALID_TO
    FROM gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MEP xc
    WHERE EVETP_SOURCE_SYS_ORIGIN = 'RDS_MEPOPERATIONTYPE'
) src
ON trg.EVETP_SOURCE_ID = src.EVETP_SOURCE_ID
    AND trg.EVETP_SOURCE_SYSTEM_ID = src.EVETP_SOURCE_SYSTEM_ID
    AND trg.EVETP_SOURCE_SYS_ORIGIN = src.EVETP_SOURCE_SYS_ORIGIN
    AND trg.EVETP_VALID_TO = src.EVETP_VALID_TO 
WHERE src.EVETP_SOURCE_ID IS NULL;

-- ============================================================================
-- STEP 7: Close Old Records in Target
-- ============================================================================

UPDATE gap_catalog.ads_owner.EVENT_TYPES 
SET EVETP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
    EVETP_UPDATE_PROCESS_KEY = CAST(p_process_key AS BIGINT),
    EVETP_CURRENT_FLAG = 'N', 
    EVETP_VALID_TO = DATE_SUB(CAST(p_load_date AS DATE), 1)
WHERE EVETP_CURRENT_FLAG = 'Y'
    AND EVETP_VALID_TO = to_date('30000101','yyyyMMdd')
    AND evetp_key||'.'||evetp_valid_from IN (
        SELECT tech_rid 
        FROM IDENTIFIER(dif_table_name) 
        WHERE tech_rid IS NOT NULL
    );

-- ============================================================================
-- STEP 8: Insert Changed Records
-- ============================================================================

INSERT INTO gap_catalog.ads_owner.EVENT_TYPES (
    EVETP_KEY, 
    EVETP_SOURCE_ID, 
    EVETP_SOURCE_SYSTEM_ID, 
    EVETP_SOURCE_SYS_ORIGIN, 
    EVETP_DESC, 
    EVETP_EVENT_TABLE_NAME, 
    EVETP_VALID_FROM, 
    EVETP_VALID_TO, 
    EVETP_CURRENT_FLAG, 
    EVETP_DELETED_FLAG, 
    EVETP_INSERTED_DATETIME, 
    EVETP_INSERT_PROCESS_KEY, 
    EVETP_UPDATED_DATETIME, 
    EVETP_UPDATE_PROCESS_KEY, 
    EVETP_TARGET, 
    EVETP_DATA_PATH
)
SELECT 
    EVETP_KEY, 
    EVETP_SOURCE_ID, 
    EVETP_SOURCE_SYSTEM_ID, 
    EVETP_SOURCE_SYS_ORIGIN, 
    EVETP_DESC, 
    EVETP_EVENT_TABLE_NAME, 
    CAST(p_load_date AS DATE) AS EVETP_VALID_FROM, 
    DATE('3000-01-01') AS EVETP_VALID_TO, 
    'Y' AS EVETP_CURRENT_FLAG, 
    tech_del_flg AS EVETP_DELETED_FLAG, 
    CURRENT_TIMESTAMP() AS EVETP_INSERTED_DATETIME, 
    CAST(p_process_key AS BIGINT) AS EVETP_INSERT_PROCESS_KEY, 
    CURRENT_TIMESTAMP() AS EVETP_UPDATED_DATETIME, 
    CAST(p_process_key AS BIGINT) AS EVETP_UPDATE_PROCESS_KEY, 
    EVETP_TARGET, 
    EVETP_DATA_PATH
FROM IDENTIFIER(dif_table_name)
WHERE tech_new_rec = 'N';

-- ============================================================================
-- STEP 9: Insert New Records
-- ============================================================================

INSERT INTO gap_catalog.ads_owner.EVENT_TYPES (
    EVETP_KEY, 
    EVETP_SOURCE_ID, 
    EVETP_SOURCE_SYSTEM_ID, 
    EVETP_SOURCE_SYS_ORIGIN, 
    EVETP_DESC, 
    EVETP_EVENT_TABLE_NAME, 
    EVETP_VALID_FROM, 
    EVETP_VALID_TO, 
    EVETP_CURRENT_FLAG, 
    EVETP_DELETED_FLAG, 
    EVETP_INSERTED_DATETIME, 
    EVETP_INSERT_PROCESS_KEY, 
    EVETP_UPDATED_DATETIME, 
    EVETP_UPDATE_PROCESS_KEY, 
    EVETP_TARGET, 
    EVETP_DATA_PATH
)
SELECT 
    EVETP_KEY_NEW + max_key AS EVETP_KEY,  -- Add the offset to maintain key sequence
    EVETP_SOURCE_ID, 
    EVETP_SOURCE_SYSTEM_ID, 
    EVETP_SOURCE_SYS_ORIGIN, 
    EVETP_DESC, 
    EVETP_EVENT_TABLE_NAME, 
    CAST(p_load_date AS DATE) AS EVETP_VALID_FROM, 
    DATE('3000-01-01') AS EVETP_VALID_TO, 
    'Y' AS EVETP_CURRENT_FLAG, 
    tech_del_flg AS EVETP_DELETED_FLAG, 
    CURRENT_TIMESTAMP() AS EVETP_INSERTED_DATETIME, 
    CAST(p_process_key AS BIGINT) AS EVETP_INSERT_PROCESS_KEY, 
    CURRENT_TIMESTAMP() AS EVETP_UPDATED_DATETIME, 
    CAST(p_process_key AS BIGINT) AS EVETP_UPDATE_PROCESS_KEY, 
    EVETP_TARGET, 
    EVETP_DATA_PATH
FROM IDENTIFIER(dif_table_name)
WHERE tech_new_rec = 'Y';

-- ============================================================================
-- STEP 10: Validation and Cleanup
-- ============================================================================

-- Display record counts for validation
SELECT * FROM (
    SELECT '1-Source_Table' AS table_name, COUNT(1) AS rec_cnt 
    FROM gap_catalog.ads_etl_owner.DLK_ADS_LOV_RDS_MEPOPERATIONTYPE 
    WHERE sys = 'Brasil'
    UNION ALL
    SELECT '2-XC_RDS_EVENT_TYPES_MEP', COUNT(1) 
    FROM gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MEP
    UNION ALL
    SELECT '3-DIFF_TABLE', COUNT(1) 
    FROM IDENTIFIER(dif_table_name)
    UNION ALL
    SELECT '4-event_types', COUNT(1) 
    FROM gap_catalog.ads_owner.event_types 
    WHERE evetp_source_sys_origin = 'RDS_MEPOPERATIONTYPE'
) ORDER BY 1;

-- Optional: Drop DIFF table if no longer needed
-- DROP TABLE IF EXISTS IDENTIFIER(dif_table_name);

-- ============================================================================
-- END OF SCRIPT
-- ============================================================================